In [ ]:
# Import der notwendigen Bibliotheken
from owslib.wms import WebMapService
import os

## Python Code zur Nutzung des WMS-Servers

In dem folgenden Abschnitt wird der Web Map Service (WMS) von Bremen verwendet, um Karten- und Geodaten zu laden und anzuzeigen. Der Code stellt eine Verbindung zum WMS-Server her, listet verfügbare Layer auf und ruft Informationen zu einem bestimmten Layer ab.

In [ ]:
# WMS-URL: Die URL, die den Web Map Service des WMS-Servers angibt.
wms_url = "https://geodienste.bremen.de/wms_dop10_2023?language=ger"

# Verbindung zum Server herstellen
wms = WebMapService(wms_url)

# Verfügbare Layer anzeigen
# Der WMS-Server enthält eine Reihe von Karten-Layern, die wir durchlaufen und ausgeben können.
# In diesem Fall listen wir die Namen der Layer und deren Titel auf.
for layer in wms.contents:
    print(layer, wms[layer].title)

# Informationen über einen bestimmten Layer abrufen
# In diesem Schritt wählen wir einen bestimmten Layer (DOP10_2023_HB) aus und holen uns seine Metadaten.
layer_name = "DOP10_2023_HB"
layer_info = wms[layer_name]
# Bounding Box des Layers: Hier erhalten wir die geographischen Grenzen (Eckkoordinaten) des Layers.
print("Bounding Box Layer:", layer_info.boundingBox)
# Unterstützte CRS (Koordinatenreferenzsysteme): Der WMS-Server kann verschiedene Koordinatensysteme unterstützen. 
# Wir listen die unterstützten CRS für den ausgewählten Layer auf.
print("Supported CRS:", layer_info.crsOptions)

## Funktion zum Herunterladen eines Bildes vom WMS-Server

In dieser Funktion wird ein Bild von einem WMS-Server heruntergeladen. Die Funktion verwendet die `getmap`-Methode des WMS-Servers, um das Bild in einem gewünschten Format und einer gewünschten Größe abzurufen und es auf der Festplatte zu speichern.

In [ ]:
def download_image(wms, layer, bbox, size, format, filename):
    response = wms.getmap(layers=[layer],   # Wählen des Layers, das heruntergeladen werden soll
                          styles=[''],      # Stil für das Layer (leer bedeutet standardmäßig)
                          srs='EPSG:25832', # Koordinatenreferenzsystem
                          bbox=bbox,        # Geographische Begrenzung (Bounding Box) des Bildes
                          size=size,        # Größe des Bildes in Pixeln (Breite, Höhe)
                          format=format,    # Format des heruntergeladenen Bildes (z.B. 'image/png')
                          transparent=True) # Option für transparente Hintergründe (True oder False)
    
    with open(filename, 'wb') as out:
        out.write(response.read())
    print(f"{filename} erfolgreich heruntergeladen.")

## Python-Code zum Herunterladen von Kartenbildern von einem WMS-Server

Dieser Code zeigt, wie man mit einem WMS-Server Bilder für ein gesamtes Gebiet oder in kleineren Kacheln herunterladen kann. Die Kacheln sind nützlich, um große Karten in handlichere Teile zu unterteilen, die leichter verarbeitet oder angezeigt werden können.

In [ ]:
# WMS-Server-URL
url = "https://geodienste.bremen.de/wms_dop10_2023"

# Verbindung zum WMS-Server herstellen
wms = WebMapService(url, version='1.3.0')

# Layer auswählen
layer = 'DOP10_2023_HB'

# Bounding Box definieren (minx, miny, maxx, maxy)
# Diese Werte geben die geographischen Grenzen des Gebiets an, für das  Kartenbilder heruntergeladen werden sollen.
bbox = (488608.7545, 5881989.4935, 490215.9319, 5883625.7616)

# Verzeichnis für die Ausgabe erstellen
output_dir = "wms_output"
os.makedirs(output_dir, exist_ok=True)

# Gesamtes Gebiet herunterladen
# Hier wird ein vollständiges Bild des angegebenen Gebiets heruntergeladen. Die Größe der Datei kann angepasst werden.
full_size = (14276, 14276) # Maximale Bildgröße des Servers: 14276x14276 Pixel
download_image(wms, layer, bbox, full_size, 'image/tiff', f"{output_dir}/full_area_tiff.tiff")
download_image(wms, layer, bbox, full_size, 'image/png', f"{output_dir}/full_area_png.png")

# Kacheln herunterladen
# Unterteilung des Gebiets in kleinere Kacheln, die einzeln heruntergeladen werden.
tile_size = 5000 # Maximale Bildgröße des Servers: 14276x14276 Pixel
num_tiles_x = 3
num_tiles_y = 3

tile_width = (bbox[2] - bbox[0]) / num_tiles_x
tile_height = (bbox[3] - bbox[1]) / num_tiles_y

for i in range(num_tiles_y):
    for j in range(num_tiles_x):
        tile_bbox = (
            bbox[0] + j * tile_width,
            bbox[1] + i * tile_height,
            bbox[0] + (j + 1) * tile_width,
            bbox[1] + (i + 1) * tile_height
        )
        
        tile_filename = f"tile_{int(tile_bbox[0])}_{int(tile_bbox[1])}"
        download_image(wms, layer, tile_bbox, (tile_size, tile_size), 'image/tiff', f"{output_dir}/{tile_filename}_tiff.tiff")
        download_image(wms, layer, tile_bbox, (tile_size, tile_size), 'image/png', f"{output_dir}/{tile_filename}_png.png")

print("Alle Bilder wurden erfolgreich heruntergeladen.")